In [ ]:
from Node import BranchingProgram
from Node import Node
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import multiA as ma

In [ ]:
k = 15
n_cells = np.zeros(2*k)
n_cells[k] = 1
lam = 1.3
for i in np.arange(1,k):
    n_cells[k + i] = n_cells[k+i-1] * lam
    n_cells[k - i] = n_cells[k - i + 1] / lam

In [ ]:
X,y = ma.load_data()
X_train, X_test, y_train, y_test = train_test_split(
    X, y, train_size=0.8, shuffle=True)

In [ ]:
print(np.shape(p_test), np.shape(p_train))

In [ ]:
def trim_arrays(A,B):
    m = np.min((len(A),len(B)))
    return A[:m],B[:m]

In [ ]:
## preparing the data, with an even/odd bias given by bias

bias = 0.9
p_train,r_train,kl = ma.prepare_sets(X_train, y_train, bias)
p_test, r_test, _ = ma.prepare_sets(X_test, y_test, bias)
p_train, r_train = trim_arrays(p_train, r_train)
true_KL = kl/np.log(2)
print("True KL (base 2) with bias",bias,"is ",kl2)

In [ ]:
# the following code is used for exporting the data to MATLAB, to run the MATLAB algorithms

mdic = {"data_p": p_train.T, "data_r": r_train.T, "data_r_test": r_test.T}
name = 'mnist_bias'+str(int(100*bias))+'.mat'
scipy.io.savemat(name, mdic)

In [ ]:
def classifier():
    return RandomForestClassifier(max_depth=5, n_estimators=10)
#     return DecisionTreeClassifier(max_depth=1)
#     return LogisticRegression(max_iter=500)

In [ ]:
me = ma.MaxEnt()
me.fit(p_train, r_train, iter=150, clr_maker=classifier, eps=0.02, eta=0.02)
me.compute_KL(r_test)

In [ ]:
def trim_arrays(A,B):
    m = np.min((len(A),len(B)))
    return A[:m],B[:m]

In [ ]:
A = BranchingProgram(n_cells, 150, 0.02, classifier, reweigh = 0)
p1,r1 = trim_arrays(p_train, r_train)
A.fit(p1, r1)
p2,r2 = trim_arrays(p_test, r_test)
A.predict_kl_pair(p2,r2)

In [ ]:
def get_digit(X,y,digit):
    Z = []
    for i in range(len(X)):
        if int(y[i])==digit:
            Z.append(X[i])
    return np.asarray(Z)

In [ ]:
X_train_digits = []
X_test_digits = []
for i in range(10):
    X_train_digits.append(get_digit(X_train, y_train, i))
    X_test_digits.append(get_digit(X_test, y_test, i))

In [ ]:
## to run the experiment for the pair of digits 8,9 select dig2=8, dig1=9

dig2 = 8
dig1 = 9
Z = np.concatenate((X_test_digits[dig2], X_test_digits[dig1]))
L2 = len(X_test_digits[dig2])
L1 = len(X_test_digits[dig1])
Zy = np.concatenate((np.zeros(L2)+dig2, np.zeros(L1)+dig1))
Z_p,Z_r,Z_kl = ma.prepare_sets(Z,Zy,0.9)
print(Z_kl/np.log(2))
data_p,data_r = trim_arrays(Z_p, Z_r)
X_test_pair_8 = data_r

In [ ]:
print(me.compute_KL(data_r))

In [ ]:
print(A.predict_kl_pair(data_p,data_r))

In [ ]:
# the following code is used for exporting the data to MATLAB, to run the MATLAB algorithms

mdic = {"digits_0": X_test_pair_0.T, "digits_1": X_test_pair_1.T, "digits_2": X_test_pair_2.T, 
       "digits_3": X_test_pair_3.T, "digits_4": X_test_pair_4.T, "digits_5": X_test_pair_5.T, 
       "digits_6": X_test_pair_6.T, "digits_7": X_test_pair_7.T, "digits_8": X_test_pair_8.T, 
       "digits_9": X_test_pair_9.T }
scipy.io.savemat('mnist_digits.mat', mdic)

In [ ]:
# the following are the results which are reported in the paper, using the cells above, 
# 'dt' here stands for decision tree of depth 1 as base classifier

mc_digits = [2.46,2.29,1.9,1.65,1.52,1.9,2.28,1.99,1.43,2]
me_digits = [2.44,1.93,2.02,1.51,1.58,2.64,2.51,1.36,1.59,2.55]
mc_dt_digits = [2.19,2.06,1.73,1.62,1.22,1.72,2.33,1.93,1.53,1.96]
me_dt_digits = [1.4,1.32,1.4,0.54,0.6,1.93,1.81,0.61,0.68,1.5]

In [ ]:
fig,ax=pl.subplots(1,1)
plt.rc('xtick', labelsize=12)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12)    # fontsize of the tick labels
plt.rc('legend', fontsize=10)    # legend fontsize
plt.rc('figure', titlesize=14)  # fontsize of the figure title
k = 10
x_axis = range(k)
ax.plot(x_axis, true_KL*np.ones(k), '--r')
ax.plot(x_axis,me_digits, color = 'b', marker='x')
ax.plot(x_axis,mc_digits, color = 'g', marker='x')
ax.plot(x_axis,me_dt_digits, '--',color = 'b', marker='+')
ax.plot(x_axis,mc_dt_digits, '--',color = 'g', marker='+')
ax.set_xticks([x for x in range(10)])
ax.set_xticklabels(x_ticks)
ax.set_ylim([0, 3.5])
x_ticks = ['\"0,1\"', '\"1,2\"', '\"2,3\"', '\"3,4\"', '\"4,5\"', '\"5,6\"', '\"6,7\"', '\"7,8\"', '\"8,9\"', '\"9,0\"']
fig.suptitle('KL divergence for subgroups for MNIST')
ax.legend(['True KL','LLK-RF5','MC-RF5','LLK-DT','MC-DT'],loc='upper left',mode = "expand", ncol = 4)
#ax.show()